### Richardson-Lucy Deconvolution with RedLionFish

For more information, please see https://github.com/rosalindfranklininstitute/RedLionfish

In [ ]:
def get_tomogram_filenames():
    # Get RLDeconv mdout
    try:
        imod_mdout = f"{os.getcwd()}/{proj_name}_rlf_deconv_mdout.yaml"
    except FileNotFoundError:
        print(
            f"{proj_name}_rlf_deconv_mdout.yaml not found, cannot find tomogram filenames."
        )

    with open(imod_mdout, "r") as f:
        output = yaml.load(f.read(), Loader=yaml.FullLoader)
        tomogram_filenames = output["raw_files"].values()
        psf_filenames = output["psf_files"].values()
        out_filenames = output["outputs"].values()

    return tomogram_filenames, psf_filenames, out_filenames

In [ ]:
def get_img_data(fname):
    # Import image data
    if fname.endswith(".mrc"):
        with mrcfile.open(fname) as mrc:
            img_data = mrc.data

    elif os.path.isdir(fname):
        volume = []
        for xy_slice in os.listdir(fname):
            volume.append(tifffile.imread(f"{fname}/{xy_slice}").data)
        img_data = np.stack(volume)

    elif fname.endswith(".tif"):
        img_data = tifffile.imread(fname)

    else:
        raise ValueError(
            "Tomogram cannot be read. Please use a 3D tiff, mrc or stack of 2D Tiffs"
        )

    return img_data

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable as mal


def get_central_slices(img_data):
    # Find central slices
    z_central = img_data.shape[0] // 2
    x_central = img_data.shape[1] // 2
    y_central = img_data.shape[2] // 2

    # Calculate number of slices to add
    zxy_add = (np.array(img_data.shape) * 0.0025).astype(int)
    zxy_add[zxy_add < 1] = 1

    # xy, yz, xz image data
    xy_data = img_data[z_central - zxy_add[0] : z_central + zxy_add[0], :, :].sum(
        axis=0
    )
    yz_data = img_data[:, x_central - zxy_add[1] : x_central + zxy_add[1], :].sum(
        axis=1
    )
    xz_data = img_data[:, :, y_central - zxy_add[2] : y_central + zxy_add[2]].sum(
        axis=2
    )

    return [xy_data, yz_data, xz_data]


def show_tomogram(tomo_fname, deconv_fname):
    """tomo_fname can be 3-D .mrc, 3-D .tiff, or stack of 2-D tiffs"""
    tomo_data = get_img_data(tomo_fname)
    deconv_data = get_img_data(deconv_fname)

    xy_tomo, yz_tomo, xz_tomo = get_central_slices(tomo_data)
    xy_deconv, yz_deconv, xz_deconv = get_central_slices(deconv_data)

    tomo_flattened = np.hstack(
        (xy_tomo.flatten(), yz_tomo.flatten(), xz_tomo.flatten())
    )
    deconv_flattened = np.hstack(
        (xy_deconv.flatten(), yz_deconv.flatten(), xz_deconv.flatten())
    )

    vmin_tomo = np.percentile(tomo_flattened, 0)
    vmax_tomo = np.percentile(tomo_flattened, 100)

    vmin_deconv = np.percentile(deconv_flattened, 0)
    vmax_deconv = np.percentile(deconv_flattened, 100)

    # Show central slices in xy, yz, xz
    fig, ax = plt.subplots(1, 2, figsize=(10, 5.5))
    fig.suptitle(os.path.basename(tomo_fname))
    title = ["x-y", "y-z", "x-z"]

    ax[0].set_title("Original")
    ax[1].set_title("Deconvolved")

    t = ax[0].imshow(xy_tomo, cmap="Greys_r", vmin=vmin_tomo, vmax=vmax_tomo)
    divider = mal(ax[0])
    cax1 = divider.append_axes("right", size="5%", pad=0.05)
    fig.colorbar(t, ax=ax[0], cax=cax1)

    d = ax[1].imshow(xy_deconv, cmap="Greys_r", vmin=vmin_deconv, vmax=vmax_deconv)
    divider = mal(ax[1])
    cax2 = divider.append_axes("right", size="5%", pad=0.05)
    fig.colorbar(d, ax=ax[1], cax=cax2)

    #     ax[0][2].imshow(
    #                 xz_tomo,
    #                 cmap="Greys_r",
    #                 vmin=vmin_tomo,
    #                 vmax=vmax_tomo
    #     )

    #     ax[1][0].imshow(
    #                 xy_deconv,
    #                 cmap="Greys_r",
    #                 vmin=vmin_deconv,
    #                 vmax=vmax_deconv
    #     )
    #     ax[1][1].imshow(
    #                 yz_deconv,
    #                 cmap="Greys_r",
    #                 vmin=vmin_deconv,
    #                 vmax=vmax_deconv
    #     )
    #     ax[1][2].imshow(
    #                 xz_deconv,
    #                 cmap="Greys_r",
    #                 vmin=vmin_deconv,
    #                 vmax=vmax_deconv
    #     )

    plt.tight_layout()

In [ ]:
# Show thumbnails of the tomograms

tomogram_filenames, psf_filenames, out_filenames = get_tomogram_filenames()
for tomo, deconv in zip(tomogram_filenames, out_filenames):
    show_tomogram(tomo, deconv)